### MPEC
Let me try Dube Fox Su (2012):
$$s_{jt}=\int_{0}^{\infty}\frac{exp\big(x'_{jt}\beta_{t}-\frac{1}{y_{i}}p_{jt}+\xi_{jt}\big)}{1+\sum_{k}exp\big(x'_{jt}\beta_{t}-\frac{1}{y_{i}}p_{jt}+\xi_{jt}\big)}\frac{1}{y_{i}\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(\log y_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)dy_{i}=
$$
$$s_{jt}=\int_{-\infty}^{\infty}\frac{exp\big(x'_{jt}\beta_{t}-\frac{1}{\exp e_{i}}p_{jt}+\xi_{jt}\big)}{1+\sum_{k}exp\big(x'_{jt}\beta_{t}-\frac{1}{\exp e_{i}}p_{jt}+\xi_{jt}\big)}\frac{1}{\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(e_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)de_{i}=
$$
$$s_{jt}=\int_{-\infty}^{\infty}\frac{1}{\sqrt\pi}\frac{exp\big(x'_{jt}\beta_{t}-\exp (-\mu_{y}-\sqrt{2}\sigma_{y}e_{i}) p_{jt}+\xi_{jt}\big)}{1+\sum_{k}exp\big(x'_{jt}\beta_{t}-\exp (-\mu_{y}-\sqrt{2}\sigma_{y}e_{i})p_{jt}+\xi_{jt}\big)}\phi(e_i)de_{i}=
$$

I prepare the data:

In [2]:
using DataFrames
using FastGaussQuadrature
using JuMP
using Ipopt

data = readtable("data_ps2.txt", header=false, separator=',')
rename!(data, names(data), [:car, :year, :firm, :price, :quantity, :weight, :hp, :ac, :nest3, :nest4]);
data[:share] = data[:quantity] / 1e8;
meanweight = mean(data[:price])
meanhp = mean(data[:price])
meanprice = mean(data[:price])

data[:weight] = data[:weight] / mean(data[:weight])
data[:hp]     = data[:hp] / mean(data[:hp])
data[:price]  = data[:price] / mean(data[:price]);

function competitor_mean_characteristic(good, char::Symbol)
    mean(data[(data[:firm] .!= good[:firm]) .* (data[:year] .== good[:year]), char]) #same year, different firm
end

function firm_mean_characteristic(good, char::Symbol)
    mean(data[(data[:firm] .== good[:firm]) .* (data[:year] .== good[:year]) .* 
            (data[:car] .!= good[:car]), char]) #same year, same firm, different product
end

data[:comp_weight] = [competitor_mean_characteristic(good, :weight) for good in eachrow(data)]
data[:comp_hp]     = [competitor_mean_characteristic(good, :hp) for good in eachrow(data)]
data[:comp_ac]     = [competitor_mean_characteristic(good, :ac) for good in eachrow(data)]

data[:firm_weight] = [firm_mean_characteristic(good, :weight) for good in eachrow(data)]
data[:firm_hp]     = [firm_mean_characteristic(good, :hp) for good in eachrow(data)]
data[:firm_ac]     = [firm_mean_characteristic(good, :ac) for good in eachrow(data)];

data[isnan.(data[:firm_weight]),:firm_weight] = 0.0
data[isnan.(data[:firm_hp]),:firm_hp] = 0.0
data[isnan.(data[:firm_ac]),:firm_ac] = 0.0;

In [3]:
year = 1990

w = convert(Array{Float64}, data[data[:year] .== year, [:share, :weight, :hp, :ac, :price]])
z = convert(Array{Float64}, data[data[:year] .== year, [:weight, :hp, :ac, :comp_weight, :comp_hp, :comp_ac,
                                                                      :firm_weight, :firm_hp, :firm_ac]]);
w = w[1:20,:]
z = z[1:20,:]

share = w[:, 1]
weight = w[:, 2]
hp = w[:, 3]
ac = w[:, 4]
price = w[:, 5];

This is the mathematical program to be solved by IpOpt:

In [4]:
# initial xis:
δ = zeros(length(weight))
(e, q) = gausshermite(30)
p = price #price
s = share #share

μ_y = getvalue(μy)
σ_y = getvalue(σy)

h(e) = 1/(√π) .* exp.(δ .+ (exp(-μ_y + 1/2 * σ_y^2) - exp(-μ_y - (√2) * σ_y * e)) .* p) / 
    (1 + sum(exp.(δ .+ (exp(-μ_y + 1/2 * σ_y^2) - exp(-μ_y - (√2) * σ_y * e)) .* p)))

error = 1
δ = zeros(length(weight))

while error > 1e-13
    δold = copy(δ)
    δ = δold .+ log.(s) .- log.(hcat(h.(e)...) * q)
    error = maximum(abs.(δ - δold))
end

LoadError: [91mUndefVarError: μy not defined[39m

In [14]:
using FastGaussQuadrature
using JuMP
using Ipopt
using NLopt

# Data
K=3
I=size(w)[1]
(e0, q0) = gausshermite(3)

# Model
#m = Model(solver=NLoptSolver(algorithm=:LD_SLSQP))
m = Model(solver=IpoptSolver(print_level=5, print_frequency_iter=1))

@variable(m, β[1:3], start = 10.0)
@variable(m, μy, start = 10)
@variable(m, σy, start = 1.0)



# incomplete: find xi from delta

@variable(m, ξ[1:I], start=0)

@objective(m, Min, (z' * ξ)' * (z' * ξ))
@NLexpression(m, integrand[i=1:I, k=1:K], exp(ξ[i] + weight[i] * β[1] + hp[i] * β[2] + ac[i] * β[3] - exp(-μy -sqrt(2) * σy * e0[k]) * price[i]) / (1 + sum((exp(ξ[j]) + weight[j] * β[1] + hp[j] * β[2] + ac[j] * β[3] - exp(-μy - sqrt(2) * σy * e0[k]) * price[j]) for j in 1:I)))

@NLconstraint(m, integral_constraint[i=1:I], (share[i]) == (sum(integrand[i,k] * q0[k] for k in 1:K)));

In [5]:
solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      500
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    10100

Total number of variables............................:       25
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Error